In [1]:
import pandas as pd
import MetaTrader5 as mt
from methods.make_single_df_from_bid_ask import make_single_df_from_bid_ask
from methods.make_dataframe_line import make_dataframe_line
# from methods.make_5min_volume_from_1_min_volume import make_5min_volume_from_1min_volume
from machine_learning_models.lstm.eurusd.lstm_predict_candle import predict_candle
# from machine_learning_models.lstm.eurusd.lstm_use_prediction import use_prediction
from machine_learning_models.lstm.lstm_collect_predictions_into_dataframe import collect_predictions_into_dataframe
# import torch

In [2]:
if not mt.initialize():
    print("initialize() failed, error code =", mt.last_error())
    quit()

account=52234337
pw='b&89zG&!PruWtP'

authorized = mt.login(account, pw)

In [3]:
# Display all rows
pd.set_option('display.max_rows', None)

# Display all columns
pd.set_option('display.max_columns', None)

# Prevent column width truncation
pd.set_option('display.max_colwidth', None)

# Display full content without wrapping
pd.set_option('display.expand_frame_repr', False)

In [4]:
TIMEFRAME_SMALL_MT = mt.TIMEFRAME_M5

In [25]:
df_from_server = make_dataframe_line(timeframe=TIMEFRAME_SMALL_MT, start_pos=0, end_pos=92)

In [26]:
len(df_from_server)

61

In [27]:
BASE_DIR = 'hist_data/'
TIME_SERIES_FOLDER = 'EURUSD/'
BID_FOLDER = '5_min/Bid/'
ASK_FOLDER = '5_min/Ask/'

BASE_DIR_LSTM = 'machine_learning_models/lstm/eurusd/pickle_files'
LSTM_MAX_VAL = BASE_DIR_LSTM + '/lstm_regressor_predict_max_values-2025-04-07.pkl'
LSTM_MIN_VAL = BASE_DIR_LSTM + '/lstm_regressor_predict_min_values-2025-04-07.pkl'
SCALER_X_MIN = BASE_DIR_LSTM + '/lstm_regressor_scaler_x_min.pkl'
SCALER_X_MAX = BASE_DIR_LSTM + '/lstm_regressor_scaler_x_max.pkl'
SCALER_Y_MIN = BASE_DIR_LSTM + '/lstm_regressor_scaler_y_min.pkl'
SCALER_Y_MAX = BASE_DIR_LSTM + '/lstm_regressor_scaler_y_max.pkl'

In [28]:
len(df_from_server)

61

In [29]:
df_joined_train, df_joined_test, df_joined_val = make_single_df_from_bid_ask(
    base_dir=BASE_DIR,
    time_series_folder=TIME_SERIES_FOLDER
)

In [30]:
print(len(df_joined_train), len(df_joined_test), len(df_joined_val))

13845 4615 4616


In [31]:
columns_for_y = [
        "open_plus_5min",
        "high_plus_5min",
        "low_plus_5min",
        "close_plus_5min",
        "open_plus_10min",    
        "high_plus_10min",
        "low_plus_10min",
        "close_plus_10min",
        "open_plus_15min",
        "high_plus_15min",
        "low_plus_15min",
        "close_plus_15min",
        "open_plus_20min",
        "high_plus_20min",
        "low_plus_20min",
        "close_plus_20min",
        "open_plus_25min",
        "high_plus_25min",
        "low_plus_25min",
        "close_plus_25min",
        "open_plus_30min",
        "high_plus_30min",
        "low_plus_30min",
        "close_plus_30min",
        "open_plus_35min",
        "high_plus_35min",
        "low_plus_35min",
        "close_plus_35min",
        "open_plus_40min",
        "high_plus_40min",
        "low_plus_40min",
        "close_plus_40min"
    ]

In [ ]:
for col in columns_for_y:
    predict_candle(df=df_joined_train, test_df=df_joined_test, validation_df=df_joined_val, base_dir=BASE_DIR_LSTM, column_for_y=col)

Epoch 80/100 | Train Loss: 0.001480 | Test Loss: 0.005780 | Validation loss: 0.003194
Epoch 90/100 | Train Loss: 0.000860 | Test Loss: 0.005107 | Validation loss: 0.001287
Epoch 100/100 | Train Loss: 0.001241 | Test Loss: 0.006599 | Validation loss: 0.004551
*******************************
After training on validation sample:
*******************************
Epoch 10/100 | Train Loss: 0.001858 | Test Loss: 0.001473 | Validation loss: 0.001139
Epoch 20/100 | Train Loss: 0.002139 | Test Loss: 0.000964 | Validation loss: 0.000832
Epoch 30/100 | Train Loss: 0.002348 | Test Loss: 0.001022 | Validation loss: 0.000748
Epoch 40/100 | Train Loss: 0.002625 | Test Loss: 0.001155 | Validation loss: 0.001152
Epoch 50/100 | Train Loss: 0.001764 | Test Loss: 0.000820 | Validation loss: 0.000623


In [ ]:
dataframe_actual = df_joined_val.tail(8)

In [ ]:
len(df_joined_val)

In [ ]:
dataframe_line = df_joined_val.iloc[4547:4608]

In [ ]:
len(dataframe_line)

In [ ]:
dataframe_predicted = collect_predictions_into_dataframe(dataframe_line=dataframe_line, base_dir_lstm=BASE_DIR_LSTM)

In [ ]:
dataframe_predicted_from_server = collect_predictions_into_dataframe(dataframe_line=df_from_server, base_dir_lstm=BASE_DIR_LSTM)

In [ ]:
dataframe_predicted_from_server

In [ ]:
dataframe_actual[['open', 'high', 'low', 'close']]

In [ ]:
dataframe_predicted

In [ ]:
# predict_min_stats(df_joined_train, df_joined_test, df_joined_val, scaler_x_dir=SCALER_X_MIN, scaler_y_dir=SCALER_Y_MIN, model_dir=LSTM_MIN_VAL)

In [ ]:
# predict_max_stats(df_joined_train, df_joined_test, df_joined_val, scaler_x_dir=SCALER_X_MAX, scaler_y_dir=SCALER_Y_MAX, model_dir=LSTM_MAX_VAL)